# Crack identification procedure

In [ ]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGrid, DICInpUnstructuredPoints, DICStateFields, \
    DICAlignedGrid, DICGrid
from bmcs_shear.dic_crack.dic_crack_list2 import DICCrackList
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');

In [ ]:
dic_points = DICInpUnstructuredPoints(U_factor=10, dir_name='B10_TV2', t=1,
                                      n_T_max=40, T_stepping='delta_T')
dic_points.read_beam_design()
dic_grid = DICGrid(dic_inp=dic_points, d_x=5, d_y=5)

In [ ]:
dsf = DICStateFields(dic_grid=dic_grid, R=8, omega_threshold=0.1, verbose_eval=True, tmodel='scalar_damage')
dsf.tmodel_.trait_set(E=1600, nu=0.18, omega_fn='exp-slope', strain_norm='Rankine', eps_max=0.01)
dsf.tmodel_.omega_fn_.trait_set(kappa_0=0.002, kappa_f=0.0028);
dcl = DICCrackList(dsf=dsf, delta_alpha_min=-np.pi/6, delta_alpha_max=np.pi/3, delta_s = 25, x_boundary=30)

In [ ]:
dcl.interact()

In [ ]:
len(dcl.cracks)

In [ ]:
self = dcl
# get the array of crack tips
x_tip_1_Ca = np.array([
    crack.X_tip_1_a for crack in self.cracks
])
# constract the array of vectors connecting each with each crack tip
diff_x_tip_1_CDa = x_tip_1_Ca[np.newaxis,:,:] - x_tip_1_Ca[:, np.newaxis,:] 
# get the distances
delta_tip_1_CD = np.linalg.norm(diff_x_tip_1_CDa, axis=-1)
# identify close crack tips below the threshold values
close_CD = delta_tip_1_CD < 30
# pick up the upper diagonal, the row index is smaller than col index
triu_row, triu_col = np.triu_indices_from(close_CD)
# select those that are close
close_rc = close_CD[triu_row, triu_col]
# realize that each crack tip is close to itself so that 
# its index will definitely appear in the active list
# Other occurrences of the same index indicate that 
# there is a close crack that needs to be skipped.
# Thus the primary cracks can be obtained using unique
# method of numpy specifying the unique positions, all other
# can be skipped
secondary_C = triu_row[close_rc] - triu_col[close_rc] < 0
secondary_D = np.unique(triu_col[close_rc][secondary_C])
prim_C = np.ones_like(self.cracks, dtype=np.bool_)
prim_C[secondary_D] = False
self.cracks[prim_C]

In [ ]:
crack_s = dcl.items['1']

In [ ]:
crack_s.dic_grid.n_T

In [ ]:
crack_s.interact()

In [ ]:
%%capture

from matplotlib import animation, rc
from IPython.display import HTML
from traitlets import HasTraits
import matplotlib.gridspec as gridspec


fig = plt.figure(figsize=(8,3))

In [ ]:
class CrackListAnimator(HasTraits):
    
    
    def init(self):
        print('init')
        n_T = dsf.dic_grid.n_T
        self.t_dic_T = np.hstack([
            np.linspace(0,1,n_T),
            np.ones((int(0.5*n_T),))
        ])
    
    def subplots(self, fig):
        gs = gridspec.GridSpec(ncols=2, nrows=1,
                               width_ratios=[3, 1],
                               #wspace=0.5,
                               hspace=0.5,
                               # height_ratios=[2, 1]
                               )
        ax_dcl = fig.add_subplot(gs[0, 0])
        ax_FU = fig.add_subplot(gs[0, 1])
#        return fig.subplots(1,1)
#        return ax_dsf#, ax_FU
        return ax_dcl, ax_FU

    def plot(self, i):
        fig.clear()
        t = self.t_dic_T[i]
        print('t', t)
        axes = self.subplots(fig)
        dcl.dsf.dic_grid.t = t
        
        ax_dcl, ax_FU = axes
#        ax_dsf = axes
#        ax_FU = axes
        
        dcl.bd.plot_sz_bd(ax_dcl)
        dcl.dsf.plot_crack_detection_field(ax_dcl, fig)
        dcl.plot_cracking_hist2(ax_dcl)
        dcl.critical_crack.plot_X_crc_t_Ka(ax_dcl, line_width=2, line_color='red', tip_color='red')
        ax_dcl.axis('equal')
        ax_dcl.axis('off');
        dcl.dsf.dic_grid.plot_load_deflection(ax_FU)
        
    def mp4_video(self):
        n_t = len(self.t_dic_T)
        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, self.plot, init_func=self.init,
                                       frames=n_t, interval=300, blit=True)
        return anim.save("cracking_animation.gif")
    
    def html5_video(self):
        n_t = len(self.t_dic_T)
        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, self.plot, init_func=self.init,
                                       frames=n_t, interval=300, blit=True)
        return anim.to_html5_video()

In [ ]:
# anim = CrackListAnimator()
# anim.init()
# anim.mp4_video()

In [ ]:
anim = CrackListAnimator()
anim.init()
with open('cracking_animation.html','w') as html_video_file:
    html_video_file.write(anim.html5_video())

In [ ]:
html_video_file = open('cracking_animation.html','r')
HTML(html_video_file.read())

Given the step length $\Delta s$ get the damage values in the distance $s$ along an angle $\alpha_\min, \alpha_\max$